In [2]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [3]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Writing reducer.py


In [4]:
%%bash

NUM_REDUCERS=8

IN_DIR="/data/wiki/en_articles_part"
TMP_DIR="wordcount_result"$(date +"%s%6N")
OUT_DIR="wordcount_result_sorted"$(date +"%s%6N")

hdfs dfs -rm -r -f -skipTrash ${TMP_DIR} > /dev/null
hdfs dfs -rm -r -f -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming word count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input ${IN_DIR} \
    -output ${TMP_DIR} > /dev/null

NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming word count sort" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -mapper "cat" \
    -reducer "sort -k2,2nr -k1,1" \
    -input ${TMP_DIR} \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -text ${OUT_DIR}/part-00000 | head -7 | tail -1

is	126420


17/10/14 20:01:55 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/14 20:01:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/14 20:01:58 INFO mapred.FileInputFormat: Total input files to process : 1
17/10/14 20:01:58 INFO mapreduce.JobSubmitter: number of splits:2
17/10/14 20:01:58 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/10/14 20:01:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1508003753639_0002
17/10/14 20:02:00 INFO impl.YarnClientImpl: Submitted application application_1508003753639_0002
17/10/14 20:02:01 INFO mapreduce.Job: The url to track the job: http://0a49ab304cbc:8088/proxy/application_1508003753639_0002/
17/10/14 20:02:01 INFO mapreduce.Job: Running job: job_1508003753639_0002
17/10/14 20:02:23 INFO mapreduce.Job: Job job_1508003753639_0002 running in uber mode : false
17/10/14 20:02:23 INFO mapreduce.Job:  map 0% reduce 0%
17/10/14 20:02:50 INFO m